In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [18]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.regularizers import l1
import pandas as pd
import time

In [19]:
def read_data(data_dir, is_train):
    data = pd.read_csv(data_dir)
    if is_train:
        data    = data.sample(frac=1).reset_index(drop=True)
        labels  = data["label"]
        labels  = to_categorical(labels)
        data    = data.drop("label", axis=1)
        data    /= 255
        return data, labels
    else:
        data /= 255
        return data

In [20]:
def build_model(layer_size, input_shape, regularizer, drop_size, num_of_classes, lr):
    model = Sequential()

    model.add(Dense(layer_size, activity_regularizer=regularizer, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_size))

    model.add(Dense(layer_size, activity_regularizer=regularizer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_size))

    model.add(Dense(num_of_classes))
    model.add(Activation("softmax"))

    optimizer = optimizers.Adam(lr=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [21]:
train_data_dir  = "/kaggle/input/digit-recognizer/train.csv"
test_data_dir   = "/kaggle/input/digit-recognizer/test.csv"

train_data, train_labels = read_data(train_data_dir, True)
test_data = read_data(test_data_dir, False)

In [22]:
layer_size      = 1024
input_shape     = [train_data.shape[1]]
regularizer     = l1(0.001)
drop_size       = 0.2
num_of_classes  = 10
lr              = 1e-5

In [23]:
model = build_model(layer_size, input_shape, regularizer, drop_size, num_of_classes, lr)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
activation_4 (Activation)    (None, 256)              

In [24]:
nb_epochs   = 100
val_split   = 0.2
batch_size  = 32

In [25]:
model.fit(train_data, train_labels, epochs=nb_epochs, validation_split=val_split, batch_size=batch_size, verbose=1)

model.save("model" + str(time.time()) + ".h5")

Epoch 1/100
1050/1050 [==============================] - 6s 5ms/step - loss: 1.8364 - accuracy: 0.5190 - val_loss: 1.0267 - val_accuracy: 0.8271
Epoch 2/100
1050/1050 [==============================] - 5s 5ms/step - loss: 1.0655 - accuracy: 0.7870 - val_loss: 0.7704 - val_accuracy: 0.8835
Epoch 3/100
1050/1050 [==============================] - 5s 5ms/step - loss: 0.8697 - accuracy: 0.8465 - val_loss: 0.6663 - val_accuracy: 0.9051
Epoch 4/100
1050/1050 [==============================] - 7s 6ms/step - loss: 0.7715 - accuracy: 0.8717 - val_loss: 0.6074 - val_accuracy: 0.9170
Epoch 5/100
1050/1050 [==============================] - 6s 6ms/step - loss: 0.7097 - accuracy: 0.8879 - val_loss: 0.5667 - val_accuracy: 0.9240
Epoch 6/100
1050/1050 [==============================] - 6s 5ms/step - loss: 0.6662 - accuracy: 0.8967 - val_loss: 0.5375 - val_accuracy: 0.9304
Epoch 7/100
1050/1050 [==============================] - 6s 5ms/step - loss: 0.6332 - accuracy: 0.9045 - val_loss: 0.5159 - val_ac

1050/1050 [==============================] - 5s 5ms/step - loss: 0.3079 - accuracy: 0.9766 - val_loss: 0.3035 - val_accuracy: 0.9710
Epoch 58/100
1050/1050 [==============================] - 5s 5ms/step - loss: 0.3045 - accuracy: 0.9783 - val_loss: 0.3023 - val_accuracy: 0.9708
Epoch 59/100
1050/1050 [==============================] - 7s 6ms/step - loss: 0.3038 - accuracy: 0.9776 - val_loss: 0.3012 - val_accuracy: 0.9714
Epoch 60/100
1050/1050 [==============================] - 7s 6ms/step - loss: 0.3013 - accuracy: 0.9774 - val_loss: 0.3005 - val_accuracy: 0.9714
Epoch 61/100
1050/1050 [==============================] - 6s 6ms/step - loss: 0.3000 - accuracy: 0.9782 - val_loss: 0.2997 - val_accuracy: 0.9712
Epoch 62/100
1050/1050 [==============================] - 5s 5ms/step - loss: 0.2961 - accuracy: 0.9796 - val_loss: 0.2985 - val_accuracy: 0.9714
Epoch 63/100
1050/1050 [==============================] - 5s 5ms/step - loss: 0.2932 - accuracy: 0.9807 - val_loss: 0.2960 - val_accuracy

In [26]:
pred    = model.predict(test_data)
preds   = pred.argmax(axis=-1)

results = pd.DataFrame(preds, columns=["Label"])
results.index +=1
results.to_csv("submission.csv", index_label="ImageId")